In [99]:
from os import path
import os
import mlrun

# Set the base project name
project_name_base = 'test'
# Initialize the MLRun environment and save the project name and artifacts path
project_name, artifact_path = mlrun.set_environment(project=project_name_base,
                                                    user_project=True)

project_path = path.abspath('./')

project = mlrun.new_project(project_name_base,
                            context=project_path,                           
                            user_project=True)
                                                    
# Display the current project name and artifacts path
print(f'Project name: {project_name}')
print(f'Artifacts path: {artifact_path}')

Project name: test-dani
Artifacts path: v3io:///projects/{{run.project}}/artifacts


In [100]:
project.set_function("code/function_invoker.ipynb", name='func_invoke')

In [105]:
%%writefile code/workflow.py
from kfp import dsl
from mlrun import mount_v3io, mlconf, load_project
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        # Add V3IO Mount
        f.apply(mount_v3io())
def kfpipeline():
    funcs['func_invoke'].spec.image = "mlrun/mlrun"
#     news_reader_deploy = funcs['func_invoke'].deploy_step()
        
    news_reader_invok = funcs['func_invoke'].as_step(params = {"endpoint" : "http://default-tenant.app.dev39.lab.iguazeng.com:32167"},
                                                     handler = "handler")

Overwriting code/workflow.py


In [106]:
project.set_workflow('main', os.path.join(os.path.abspath(project.context), 'code', 'workflow.py'))

In [107]:
project.save(os.path.join(project.context, 'project.yaml'))

In [108]:
project.run('main', artifact_path=artifact_path, dirty=True)

> 2021-07-13 11:17:37,826 [info] Pipeline run id=794decf0-288f-4b68-9cb6-8a9c6cb80ea9, check UI or DB for progress


'794decf0-288f-4b68-9cb6-8a9c6cb80ea9'

In [50]:
import mlrun.feature_store as fs

In [74]:
mlrun.set_environment(project="stocks-dani")

('stocks-dani', 'v3io:///projects/{{run.project}}/artifacts')

In [75]:
df = fs.get_offline_features('stocks-vec').to_dataframe()

In [76]:
df.symbol.unique()

array(['INTC', 'MSFT', 'AMZN', 'GOOGL', 'AAPL'], dtype=object)

In [79]:
df.sort_values(by=["Datetime"],ascending=False)["Datetime"][0]

Timestamp('2021-07-07 13:30:00')

In [88]:
df

,symbol,Datetime,Opens_min_1h,Opens_max_1h,Volumes_min_1h,Volumes_max_1h,Open,High,Low,Close,Volume,Content,Link,Sentiment
0,INTC,2021-07-07 13:30:00,56.369999,56.369999,365754.0,365754.0,56.369999,56.389999,56.165001,56.180000,365754,"Non-Fungible Tokens, or NFTs, are widely succe...",https://www.investing.com/news/cryptocurrency-...,1.000000
1,MSFT,2021-07-07 13:30:00,279.399994,279.399994,778863.0,778863.0,279.399994,279.950012,279.359985,279.549988,778863,NaN,NaN,NaN
2,INTC,2021-07-07 13:30:00,56.369999,56.369999,365754.0,365754.0,56.369999,56.389999,56.165001,56.180000,365754,"Non-Fungible Tokens, or NFTs, are widely succe...",https://www.investing.com/news/cryptocurrency-...,1.000000
3,INTC,2021-07-07 13:30:00,56.369999,56.369999,365754.0,365754.0,56.369999,56.389999,56.165001,56.180000,365754,"Non-Fungible Tokens, or NFTs, are widely succe...",https://www.investing.com/news/cryptocurrency-...,1.000000
4,INTC,2021-07-07 13:30:00,56.369999,56.369999,365754.0,365754.0,56.369999,56.389999,56.165001,56.180000,365754,"Non-Fungible Tokens, or NFTs, are widely succe...",https://www.investing.com/news/cryptocurrency-...,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219790,MSFT,2021-07-12 19:59:00,276.980011,277.709900,16693.0,363763.0,277.274994,277.429993,277.140015,277.320007,363763,By Yasin Ebrahim\nInvesting.com – The S&P 500 ...,https://www.investing.com/news/stock-market-ne...,0.000000
219791,AAPL,2021-07-12 19:59:00,144.428604,144.768997,46042.0,917328.0,144.449997,144.610001,144.440002,144.500000,917328,By Yasin Ebrahim\nInvesting.com – The S&P 500 ...,https://www.investing.com/news/stock-market-ne...,0.000000
219792,GOOGL,2021-07-12 19:59:00,2532.620117,2541.314209,486.0,16725.0,2539.290039,2539.620117,2537.030029,2537.979980,16725,Shares of Newegg Commerce (NEGG) were up big l...,https://www.investing.com/news/stock-market-ne...,-0.333333
219793,AMZN,2021-07-12 19:59:00,3711.800049,3717.790039,803.0,30218.0,3716.989990,3718.520020,3715.989990,3718.520020,30218,By Dhirendra Tripathi\nInvesting.com -- Invest...,https://www.investing.com/news/stock-market-ne...,-2.333333
